In [ ]:
import pandas as pd
import numpy as np
import pyodbc as db
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
# #use this script when DB updated
# conn = db.connect('Driver={SQL Server};'

# 'Server=your server;'

# 'Database=your db;'

# 'Trusted_Connection=yes;')


# query = """your query"""
# instead of this you use csv or any other data.

# cursor = conn.cursor()

# maindata = pd.read_sql_query(query,conn)

# conn.close()

# maindata.to_csv('tb_customer_alltime', index=False)

In [ ]:
data = pd.read_csv('tb_customer_alltime')

In [ ]:
data.head()

In [ ]:
data[data['Adet'] < 0]

In [ ]:
data = data[data['Adet'] > 0]

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()

In [ ]:
data.info()

In [ ]:
data.describe() 

# RFM

In [ ]:
monetary = data.groupby('master_id')['ToplamAV'].sum()
monetary = monetary.reset_index()
monetary.head()

In [ ]:
frequency = data.groupby('master_id')['Adet'].sum()
frequency = frequency.reset_index()
frequency.head()

In [ ]:
data[data['master_id']== '305318806141950919614807404831612078112-A']

In [ ]:
recency = data.groupby('master_id')['SonAV'].min()
recency = recency.reset_index()
recency.head()

In [ ]:
rfm = pd.merge(recency, frequency, on='master_id', how='inner')

In [ ]:
rfm = pd.merge(rfm, monetary, on='master_id', how='inner')

In [ ]:
rfm.columns = ['master_id','Recency', 'Frequency', 'Monetary']

In [ ]:
rfm

In [ ]:

rng = range(0, int(rfm['Monetary'].max()))
rng

In [ ]:
attributes = rfm[['Recency', 'Frequency', 'Monetary']]
attributes.boxplot()

In [ ]:
rfm[['Recency', 'Frequency', 'Monetary']]

In [ ]:

# sns.set_style("whitegrid")
  
# sns.boxplot(x = rfm[['Recency', 'Frequency', 'Monetary']], data = rfm)

In [ ]:
scaler = StandardScaler()
rfm_norm = rfm[['Recency', 'Frequency', 'Monetary']]
rfm_norm

In [ ]:
rfm_norm =scaler.fit_transform(rfm_norm)
rfm_norm = pd.DataFrame(rfm_norm)
rfm_norm

In [ ]:
ssd = []
for i in range(1,15):
    kmeans = KMeans(n_clusters=i,init= 'k-means++', random_state=42)
    kmeans.fit(rfm_norm)
    ssd.append(kmeans.inertia_)

In [ ]:
#Elbow curve
plt.figure(figsize=(10,6))
plt.plot(range(1,15),ssd)
plt.plot(range(1,15),ssd,linewidth=2,color='red',marker='8')
plt.xlabel('K Value')
plt.xticks(np.arange(1,15,1))
plt.ylabel('SSD')
plt.show()

In [ ]:
# for i in range(4,8):
#     kmeans = KMeans(n_clusters=i, max_iter=50,random_state=42)
#     kmeans.fit(rfm_norm)

#     cluster_labels = kmeans.labels_

#     silhouette_avg = silhouette_score(rfm_norm, kmeans.labels_)
#     print("For n={0} cluster, the silhoute score is {1}".format(i,silhouette_avg))

In [ ]:
sse_ = []
ssd = []
centroids_from_kmeans = []
iterations = []

for k in range(2, 15):
    kmeans = KMeans(n_clusters= k, init= 'random', n_init= 100, max_iter= 500, tol= 1e-10).fit(rfm_norm)
    sse_.append([k, silhouette_score(rfm_norm, kmeans.labels_)])
    ssd.append([k, kmeans.inertia_])
    centroids_from_kmeans.append(kmeans.cluster_centers_)
    iterations.append([k, kmeans.n_iter_])

print("Silhouette Score for each K : \n", sse_)
print("Inertia for each K : \n", ssd)
print("Number of iterations for each K : ", iterations)